In [1]:
!pip install thop
!pip install filesplit;

In [2]:
from filesplit.merge import Merge

merge = Merge('splitted_checkpoints', 'checkpoints', 'best_checkpoint.pth')
merge.merge()

In [6]:
import os
import csv
from matplotlib.dates import _epoch
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision import models
from models.network_scunet import SCUNet

# Initialize model
#model = SCUNet(in_nc=1, config=[4, 4, 4, 4, 4, 4, 4], dim=64)
# Now, we are using this config with extended dataset i.e., JNU, BUS and XPIE dataset.
SCUNet()
model = SCUNet(in_nc=1, config=[2,2,2,2,2,2,2], dim=64)

# Initialize checkpoint epoch.
# checkpoint_epoch = 2000
# checkpoint = torch.load('checkpoints/epoch_' + str(checkpoint_epoch) + '_checkpoint.pth')
# Load best model checkpoint
checkpoint = torch.load('checkpoints/best_checkpoint.pth')
model.load_state_dict(checkpoint['state_dict'])

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()  # Set model to evaluation mode

# Function to remove padding
def remove_padding_val(img_tensor, pad_sizes):
    pad_w_left, pad_w_right = pad_sizes[0], pad_sizes[1]  # padding widths
    pad_h_top, pad_h_bottom = pad_sizes[2], pad_sizes[3]  # padding heights
    h, w = img_tensor.shape[2], img_tensor.shape[3]
    cropped_img = img_tensor[:, :, pad_h_top:h - pad_h_bottom, pad_w_left:w - pad_w_right]
    return cropped_img

# Initialize noise and blur levels.
# gaussian_noise_level = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08,
# 0.09, 0.10]
# speckle_noise_level = [1, 3, 5, 7, 10, 13, 15, 17, 20, 23, 25]
# blur_level = [0, 1, 3, 5, 7, 9, 11, 13, 15]
test_noise_mode='speckle'
test_noise_level=25
test_blur_level=0

# print(f'Noise level: {test_noise_level}')
# Validation/Test dataloader
original_dir = 'datasets/XPIE/object'
filtered_dir = 'datasets/XPIE/object'
dataset_name = "XPIE_Object_test_dataset"
from DenoisingXPIETestDataset import *
XPIE_test_dataset = DenoisingXPIETestDataset(original_dir, filtered_dir, \
                                            phase='test', target_filter='', \
                                            test_noise_level=test_noise_level, \
                                            test_blur_level=test_blur_level, \
                                            test_noise_mode=test_noise_mode)
test_loader = DataLoader(XPIE_test_dataset, batch_size=1, shuffle=False, )

results_dir = os.path.join('results', dataset_name, 'best_checkpoint_epoch', \
                            'test_noise_mode_' + test_noise_mode + '_test_noise_level_' + \
                            str(test_noise_level) + '_test_blur_level_' + str(test_blur_level))

os.makedirs(results_dir, exist_ok=True)

# Loop through test dataset
with torch.no_grad():
    for idx, (original_images, degraded_images, filtered_images, pad_sizes) in enumerate(test_loader):
        original_images = original_images.float().to(device)  # Convert to float
        degraded_images = degraded_images.float().to(device)
        filtered_images = filtered_images.float().to(device)

        # Model inference
        original_outputs = model(original_images)
        outputs = model(degraded_images)
        # print(original_images.shape, outputs.shape, pad_sizes)
        outputs = remove_padding_val(outputs, torch.stack(pad_sizes))
        # print(original_images.shape, outputs.shape)
        degraded_images = remove_padding_val(degraded_images, pad_sizes)
        original_images = remove_padding_val(original_images, pad_sizes)
        # print("Images Shape: ", original_images.shape, outputs.shape, pad_sizes)


        # Save images as PNG files
        original_input_img_path = os.path.join(results_dir, f'original_input_{idx}.png')
        degraded_input_img_path = os.path.join(results_dir, f'degraded_input_{idx}.png')
        original_output_img_path = os.path.join(results_dir, f'original_output_{idx}.png')
        denoised_output_img_path = os.path.join(results_dir, f'denoised_output_{idx}.png')
        target_img_path = os.path.join(results_dir, f'target_{idx}.png')
        # print("Squeezed Image", original_images.squeeze(0).shape)
        # print(original_images)
        save_image(original_images.squeeze(0), original_input_img_path)  # Save original input image
        save_image(degraded_images.squeeze(0), degraded_input_img_path)  # Save degraded input image
        save_image(original_outputs.squeeze(0), original_output_img_path)         # Save original output image from non degraded input
        save_image(outputs.squeeze(0), denoised_output_img_path)         # Save output (denoised) image
        save_image(filtered_images.squeeze(0), target_img_path) # Save target image

Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.000000
Block Initial Type: W, drop_path_rate:0.000000
Blo